# Designing New 🌈 Features

The `Rainbow` class is the heart of `chromatic`. We aim for it to be as intuitive and easy-to-use as possible, to enable transparent and repeatable analysis of spectrosopic time-series datasets. This page collects a few explanations that might be useful if you're trying to develop new  `Rainbow` features.

In [ ]:
from chromatic import *

## How are `Rainbow` objects organized? 

The user mainly interacts with quantities like time, wavelength, and flux via the `.time`, `.wavelength`, `.flux` attributes. However, if you more closely at what's happening inside a `Rainbow` object, you'll see that these are properties that pull data from some core dictionaries. The general user probably doesn't need to interact with the core dictionaries, but if you're developing new features for `Rainbow` objects you will want to understand what's happening a little more clearly.

The core dictionaries are designed to store quantities that have dimensions like either `wavelength`, `time`, or `flux`:
- `.wavelike[...]` contains everything that has the same dimensions as `wavelength`. This dictionary will contain at least a `'wavelength'` key, with the actual wavelengths themselves. It might *also* contain information like the average spectrum of the star, the average S/N at each wavelength, the number of original detector pixels that wound up in this particular wavelength bin, and/or a mask of what wavelengths should be considered good or bad (no matter the time point). 
- `.timelike[...]` contains everything that has the same dimensons as `time`. This dictionary will contain at least a `'time'` key, with the actual times themselves. It might *also* contain information like a broadband light curve of the transit, the x or y position of the spectrum on the detector, the temperature of the detector, and/or a mask of what times should be considered good or bad (no matter the wavelength).
- `.fluxlike[...]` contains everything that has the same dimensons as `flux`. This dictionary will contain at least a `'flux'` key, with the actual fluxes themselves. It should also contain an `'uncertainty'` keyword with the uncertainties associated with those fluxes (or maybe `None`). It might *also* contain information about other quantities that depend on both time and wavelength, such as the centroid of the spectral trace, the maximum fraction of saturation, and/or a mask of what individual points should be considered good or bad.

There is one more core dictionary:
- `.metadata` contains general information that might be useful to hang onto, to pass along to another derived object, or to save out to a file. 

In [ ]:
r = SimulatedRainbow().inject_noise()

In [ ]:
r.wavelike.keys()

In [ ]:
r.timelike.keys()

In [ ]:
r.fluxlike.keys()

In [ ]:
r.metadata.keys()

When you retrieve variables with something like `.wavelength`, `.time`, `.flux`, data is being pulled directly from these dictionaries. 

## What can `Rainbow` objects do? 

We defined a small lexicon of things that `Rainbow` objects can do. If you want to write a new feature, hopefully it fits into one of these categories. If not, we can certainly discuss adding new ones!

- **actions** return a new `Rainbow` object as the result. As such, they can be chained together into commands like `r.bin(R=5).normalize().plot()`.
- **visualizations** create some graphical representation of the data in a `Rainbow`.
- **wavelike_summaries** return a wavelike-shaped summary of a `Rainbow`, with one quantity for each wavelength.
- **timelike_summaries** return a timelike-shaped summary of a `Rainbow`, with one quantity for each time.

Each of these categories has its own directory inside `chromatic/rainbows/` where the corresponding code should be stored. The `.help()` method attached to any `Rainbow` will list everything you can do with it. 

In [ ]:
r.help()

## How do we add new abilities to `Rainbow` objects?

If you want to add a new method to a `Rainbow` object, there are a few general steps you'll probably want to follow.

1. Install in development mode (see [Installation](../installation)), so you can modify the code package directly and test the code in place.
1. Decide a category of "things a `Rainbow` can do" in which it belongs.
1. Find the directory for that category in the `chromatic` code package. For example, "actions" that return new `Rainbow` objects will be in `chromatic/rainbows/actions/`. 
1. Look at another example in that directory to get a sense for the general layout. For example, you'll notice that "actions" generally create a copy of `self`, make some changes to that copy, and then return it as the new object. 
1. Add your new function, either to an existing `.py` file where it would make sense or to its own new `.py` file. The first argument to your function should be `self`, which is the `Rainbow` object itself, and then any additional arguments should follow afterward. A good way to test out your new function in a notebook or a small isolated script might look something like the following:
```python
from chromatic import *
r = SimulatedRainbow().inject_noise()
def snazzy_new_action(self, x=2):
    
    # create a copy of the original object
    new = self._create_copy()
    
    # do something
    new.fluxlike['flux'] = new.flux**x
    
    # return the modified copy
    return new
output = snazzy_new_action(r)
```
This allows you to develop and test your new function without having to worry about it being imported properly into the core `Rainbow` definition. 
1. Connect your new function into the `Rainbow` class definition. Normally, we might define a new method directly in the same file as the class definition itself, but we wanted to split the method definitions into multiple files and directories, to make things easier to find. Importing your new function to become a `Rainbow` method takes a few steps. For example, let's imagine you're making a new "action" called `snazzy_new_action` and it's located in `chromatic/rainbows/actions/snazzy.py`:
    - In `rainbows/actions/snazzy.py`, include line of code like `__all__ = ['snazzy_new_action']` at the top. The `__all__` list defines what would get imported via a line like `from chromatic.rainbows.actions.snazzy import *` (things not in `__all__` will need to be explicitly imported). 
    - In `rainbows/actions/__init__.py`, include a line of code like `from .snazzy import *`, so that imports from `chromatic.rainbows.actions` will know how to find things in `snazzy.py`. 
    - In `rainbows/rainbow.py`, down at the very bottom of the class definition for `Rainbow`, add `snazzy_new_action` to the list of `from .actions import (...)`. This, finally, will mean that all `Rainbow` objects will have access to your new method, and we can do things like `r.snazzy_new_action()` from any `Rainbow`.
    - In `rainbows/actions/descriptions.txt`, add a row describing your snazzy new action. This table defines what appears in the `.help()` method.
1. Write a test for your new feature. This is a function that somehow tests whether your new feature works; the simplest form would be "does this function run", a slightly fancier version would be "are its outputs accurate." To write a test:
    - Look in `chromatic/tests/` to find a bunch of `test_*.py` files with examples of automated tests in them. 
    - Create a function that has `*test*` somewhere in its name and store it somewhere in the `tests` directory. At a minimum, this function should test that the bit of code you wrote runs without breaking. 
    - Run `pytest` from the command line within the main repository directory. This will run your test function (along with all the other tests) and tell you whether it passed or failed. Make sure it passes!

## How do we add new readers/writers for `Rainbow` objects?

You might want to create a new reader or writer, to allow `chromatic` to interact with your own datasets or tools. To facilitate this, templates are available with human-friendly instructions for how to add a new reader or writer. If you want to try to incorporate a new reader or writer, please:
1. Install in development mode (see [Installation](../installation)), so you can modify the code package directly and test the code in place.
2. Navigate to `chromatic/rainbows/[readers|writers]/template.py`.
3. Follow the instructions in the comments in that file.

Good luck! 